In [1]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.data.path.append('.')

# Part 1: Load and Preprocess Data

## Part 1.1: Load the data

In [2]:
with open("en_US.twitter.txt", "r", encoding="utf8") as f:
    data = f.read()

## Part 1.2 Pre-process the data

In [3]:
def split_to_sentences(data):

    sentences = data.split ('\n')

    # drop leading and trailing spaces
    sentences = [s.strip() for s in sentences]
    # drop sentences if they are empty strings
    sentences = [s for s in sentences if len(s) > 0]

    return sentences

In [4]:
def tokenize_sentences (sentences):

    tokenized_sentences = []

    for sentence in sentences:

        sentence = sentence.lower()
        tokenised = nltk.word_tokenize(sentence)

        tokenized_sentences.append(tokenised)
    
    return tokenized_sentences

In [5]:
def get_tokenized_data (data):

    sentences = split_to_sentences(data)

    tokenized_sentences = tokenize_sentences(sentences)

    return tokenized_sentences

Split into train and test sets

In [6]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [7]:
def count_words (tokenize_sentences):

    word_counts = {}

    for sentence in tokenize_sentences:

        for token in sentence:

            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    
    return word_counts

Handling 'Out of Vocabulary' words

In [8]:
def get_words_with_nplus_frequency (tokenize_sentences, count_threshold):

    closed_vocab = []

    word_counts = count_words(tokenize_sentences)

    for word, cnt in word_counts.items():

        if cnt >= count_threshold:

            closed_vocab.append(word)
    
    return closed_vocab

In [9]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):

    vocabulary = set(vocabulary)

    replaced_tokenized_sentences = []

    for sentence in tokenized_sentences:

        replaced_sentence = []

        for token in sentence:

            if token in vocabulary:
                replaced_sentence.append(token)
            else:
                replaced_sentence.append(unknown_token)

        replaced_tokenized_sentences.append(replaced_sentence)

    return replaced_tokenized_sentences

In [10]:
def preprocess_data (train_data, test_data, count_threshold):

    vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)

    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)

    return train_data_replaced, test_data_replaced, vocabulary

Preprocess the train and test data

In [11]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data, minimum_freq)

# Part 2: Develop n-gram based language models

In [71]:
def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):

    n_grams = {}

    for sentence in data: 
        
        sentence = [start_token] * n+ sentence + [end_token]

        sentence = tuple(sentence)
        
        for i in range(len(sentence) if n==1 else len(sentence)-1): 
        
            n_gram = sentence[i:i+n]
            
            if n_gram in n_grams.keys(): 
            
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    
    return n_grams

In [75]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    previous_n_gram = tuple(previous_n_gram)

    if previous_n_gram in n_gram_counts:
        previous_n_gram_count = n_gram_counts[previous_n_gram]
    else:
        previous_n_gram_count = 0

    denominator = previous_n_gram_count + k * vocabulary_size

    n_plus1_gram = previous_n_gram + (word,)

    if n_plus1_gram in n_plus1_gram_counts:
        n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram]
    else:
        n_plus1_gram_count = 0

    numerator = n_plus1_gram_count + k

    probability = numerator / denominator

    return probability

Estimate probabilities for all words

In [78]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):

    previous_n_gram  = tuple(previous_n_gram)

    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)

    probabilities = {}

    for word in vocabulary:

        prob = estimate_probability(word, previous_n_gram, 
                                           n_gram_counts, n_plus1_gram_counts, 
                                           vocabulary_size, k=k)        
        probabilities[word] = prob
    
    return probabilities

Count and probability matrices

In [83]:
def make_count_matrix (n_plus1_gram_counts, vocabulary):

    vocabulary = vocabulary + ["<e>", "unk"]

    n_grams = []

    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    
    n_grams = list(set(n_grams))

    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}

    col_index = {word:j for j, word in enumerate(vocabulary)}

    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))

    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
    
    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix


In [85]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

# Part 3: Perplexity

In [89]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    n = len(list(n_gram_counts.keys())[0])

    sentence = ["<s>"] * n + sentence + ["<e>"]
    sentence = tuple(sentence)
    N = len(sentence)
    
    product_pi = 1.0

    for t in range (n, N):

        n_gram = sentence[t-n:t]

        word = sentence[t]

        probability = estimate_probability(word,n_gram, n_gram_counts, n_plus1_gram_counts, len(unique_words), k=1)

        product_pi *= 1/ probability

    perplexity = product_pi ** (1/float(N))

    return perplexity


# Part 4: Build an auto-complete system

In [91]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):

    n = len(list(n_gram_counts.keys())[0])

    previous_n_gram = previous_tokens[-n:]

    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    
    suggestion = None
    max_prob = 0

    for word, prob in probabilities.items():

        if start_with != None:

            if not word.startswith(start_with):
                continue
        
        if prob > max_prob:

            suggestion = word
            max_prob = prob

    return suggestion, max_prob

# part 5 : Results

In [92]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
# test your code when setting the starts_with
tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")


The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `cat` with a probability of 0.0909
